In [15]:
!pip install googletrans

In [28]:
import os
import sys
import pandas as pd
import googletrans
from googletrans import Translator
%matplotlib inline
import matplotlib.pyplot as plt

In [24]:
DIR_PATH = r"C:\Users\gabi9\Desktop\Dev\python_dev\elections"
CALPI = "calpi"
SETTELMENT = "settelments"
translator = Translator()



def get_data(agg_type, num):
    """returns dataframe of the requested csv
    
    parameters
    ----------
    agg_type - aggregation type of the csv needed
    num - election number
    
    return
    ------
    DataFrame
    """
    if type(num) is not str:
        num = str(num)
    
    ROOT_PATH = os.path.abspath(os.getcwd())
    path = os.path.join(ROOT_PATH,num+agg_type+".xlsx")
    df = pd.read_excel(path)
    df_en = df.copy()
    
    df_en.rename(columns=lambda x: translator.translate(x,dest='en',src='he').text, inplace=True)
    return df_en
    

In [29]:
CALPI = "calpi"
SETTELMENT = "settelments"

In [129]:
def getdata(agg_type , num):
    
    if type(num) is not str:
        num = str(num)
    ROOT_PATH = os.path.abspath(os.getcwd())
    path = os.path.join(ROOT_PATH,num+agg_type+".xlsx")
    df = pd.read_excel(path)
    return df

def getRideofColm(df):
    dict = {}
    for column in df.columns[6:]:
       dict[column]=df[column].sum()
    for c in dict:
        if dict[c] < 10000:
           df = df.drop(c, 1)
    for i,r in df.iterrows():
        for c in df.columns[6:]:
            x = r[c]/r['valid votes']*100
            df.at[i , c]=x

    return df
        


    
df21=getdata(SETTELMENT,21)
df21=getRideofColm(df21)
print(df21)

#plt.figure(1)
#plt.plot(df21['City name'], df21['Number of polling'], color='b')
#plt.xlabel('City name')
#plt.ylabel('Number of polling')
#plt.title('')

       code                  name  Holders of voting rights  Voters  \
0         7                SHAHAR                       516     371   
1        10                TIROSH                       333     237   
2        11               NIR HEN                       403     289   
3        13                HAZEVA                       502     403   
4        15                 NO'AM                       302     217   
...     ...                   ...                       ...     ...   
1209   9500         QIRYAT BIALIK                     34220   22168   
1210   9600            QIRYAT YAM                     36506   21114   
1211   9700          HOD HASHARON                     45512   34904   
1212   9800  BINYAMINA-GIV'AT ADA                     10073    7652   
1213  99999        EXTERNAL_VOTES                         0  240783   

      Disqualified  valid votes  Avoda  United Torah Judaism  UAL-Balad  \
0                0          371      4                     0          0 